In [ ]:
import sys
import os
os.environ['PROJ_LIB']='/anaconda3/share/proj'
sys.path.append('/Users/sol/Google Drive/professional/research/utilities/python_myfun')
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.basemap import Basemap
from map_mymod import *
import datetime
import geopy.distance

In [ ]:
# storm to analyze
stnyr, stn='2006055','055' 
#stnyr, stn='2007050','050' 
# storm track data
fn2='/Users/sol/Documents/LargeData/tc_wakes/storms_Data/'+stnyr+'/'+stn+'annual_storm_info.nc'
storm=xr.open_dataset(fn2)
storm.close()
# transform storm time to comparable time to data
date_1858 = datetime.datetime(1858,11,17,0,0,0) # start date is 11/17/1958
storm_date=[0]*len(storm.time.values[0]) 
for i in range(len(storm.time.values[0])):
    storm_date[i]=date_1858+datetime.timedelta(days=float(storm.time.values[0][i]))  
# transform storm lon to -180:180
storm.lon[0]=[i-360 for i in storm.lon[0] if i>=180]

In [ ]:
# gather data from around the storm
# open file
fn='/Users/sol/Documents/LargeData/tc_wakes/storms_Data/'+stnyr+'/'+stn+'_combined_data.nc'
dtd=xr.open_dataset(fn)
dtd.close()
#print(dt)
#nmask=dt.storm_mask.where(dt.storm_mask<0)*-1
nmask=dtd.dist_from_storm_km.where(dtd.dist_from_storm_km<600)
#we also want the land mask
nmask=nmask*0+1
nmask=nmask*dtd.mask[0,:,:]

In [ ]:
# for each point (of data within the storm mask) get data into two structures: sif (all points), adfs (array)
tp=0
rw=[] # extra info array
ar=np.full([10000,len(dtd.time),4],np.nan)
for i in range(len(dtd.lat.values)):
    for j in range(len(dtd.lon.values)):
        if not np.isnan(nmask[i,j]):
            #print(i,j)
            #plt.plot(dt.lon[j],dt.lat[i],'b.-')
            
            # lat lon for each point in the mask
            tla=dtd.lat[i].values
            tlo=dtd.lon[j].values
            di=dtd.dist_from_storm_km[i,j].values
            # timeseries data (3d: point, t, [sst, u, v])
            # all time for data series
            ar[tp,:,0]=dtd.analysed_sst[:,i,j].values-273.15
            ar[tp,:,1]=dtd.uwnd[:,i,j].values
            ar[tp,:,2]=dtd.vwnd[:,i,j].values
            ar[tp,:,3]=dtd.dbss_obml[:,i,j].values # maybe redo for previous month
            
            rw.append([tla,tlo,di])
            tp=tp+1

#print(tp, len(rw))
tp=tp-1 # last index
# convert info to dataframe and save
cols=['lat','lon','dist_km']
sif=pd.DataFrame(rw,columns=cols)
#sif.to_csv('../data/storm_sum_data/info_locdist_'+stnyr+'.csv')

# convert data to xarray
ar=np.delete(ar,range(tp+1,10000),axis=0)
dt=xr.DataArray(ar,coords=[range(tp+1),dtd.time,['SST','uwnd','vwnd','mld']],dims=['location','time','var']) 
#adfs.to_netcdf('../data/storm_sum_data/stormdatamask_'+stnyr+'.nc')
dttime=pd.to_datetime(dt.time.values) # datatime array

In [ ]:
def get_dist_grid(lat_point,lon_point,lat_list,lon_list):
    # adapted from chelle's routine
    #this routine takes a point and finds distance to all points in a grid of lat and lon
    dist_grid = np.empty((len(lat_list)))    
    coords_1 = (lat_point, lon_point)  
    for i in range(len(lat_list)):
        coords_2 = (lat_list[i], lon_list[i])  
        arclen_temp = geopy.distance.geodesic(coords_1, coords_2).km  #distance in km   
        #print(arclen_temp)
        dist_grid[i]=arclen_temp
    return dist_grid

In [ ]:
def find_storm_beginend(pdt): # a point in sif
    # find begining and end of storm influence
    thwnd=5 # threshold of wind speed to consider significant storm
    thdist=600 # threshold of distance strom traveled between two day to consider it close
    
    # calculate speed for all times of pdt
    spd=np.sqrt(dt[pdt,:,1]*dt[pdt,:,1]+dt[pdt,:,2]*dt[pdt,:,2])
    
    # calculate distance from point pdt to each storm track point 
    nd=get_dist_grid(sif['lat'][pdt],sif['lon'][pdt],storm.lat.values[0],storm.lon.values[0])
    
    # find min distance from storm to point and time
    clod=min(nd) # closest distance to storm
    a=np.where(nd==clod)
    b=a[0][0] # index of closest distance on storm track
    ndi=np.abs(dttime - storm_date[b]).argmin() # find index of time on data where storm is closest
    
    # choose closest as first day of strom to begin
    begs=ndi # index on data for closest point to storm
    tb=b # Storm time index of closest
    # look for the days before to see if storm arrived earlier (strong winds and close storm)
    
    # find distance to storm at day before  
    if tb>2: # only days when storm closest distance is later than the first few storm points
        if spd[begs]<=thwnd: # wind speed of closest is below threshold end routine
            ok=0 # end search for beginning and do not search for end
            ends=begs
        else: # look for begining day before 
            ok=0
            while ok==0:
                if tb>2: # there is a previous storm day
                    if spd[begs-1]<=thwnd: # if previous wind is below threshold end routine
                        # the day begs is the real begining, but we have found the begining
                        ok=1 
                    else: # check distance 
                        # calculate distance to storm at begs-1
                        # fist calculate storm time closest to begs-1
                        #tb=np.abs(np.array(storm_date)-dttime[begs-1]).argmin() 
                        #if np.logical_and(b-tb>3,tb>2): # making sure that the begs-1 is still a storm day (and a real day before the closest)
                        if tb-4>0:
                            dts=nd[tb-4] # distance to storm at begs-1 time
                            if dts>=thdist: # if storm was too far the day before, then wind is probably not due to the storm
                                ok=1 # leave begs as the begining of storm
                            else: # if the storm was close the day before, keep lookin
                                begs=begs-1
                                tb=tb-4
                        else: # no room to go back
                            ok=1
                else: # there is no prveious strom date
                    ok=1
                    
                
    else: # point is to close to the begining of the strom - begin assumed the closest day
        ok=1 # look for end of storm
    
    if ok==1: # check for the end of the storm 
        
        # end of event when storm ends
        endstorm=np.abs(dttime - storm_date[len(storm_date)-1]).argmin()
        ok=0
        ends=begs+1 # end of strom influence
        while np.logical_and(ok==0,ends<=endstorm):
            if nd[ends]>thdist: # Storm too far away
                ok=1
            else:
                if spd[ends]<thwnd:
                    ok=1
                else:
                    ends=ends+1
        
    #print(nd[tb],spd[begs])
 
    return begs, ends, clod

In [ ]:
def cold_wake(pn,nidx, nfdx):
    # find and quantify cold wake
    # pass point, day (index) of storm arrival, and end of wind forcing (storm)
    # returns: dcwi0 (beg of coldwake),lencw (lenght), sst0 (base temp), mxdT (max cooling), mxdTi (day of max cool, relative to cold wake beg)
    # returns also: mld (mix layer depth at begining of storm)
    
    # within the 2 next days (realistic?)
    # needs fixing to consider increases in wind
    cow=0 # is there a cold wake? 0 = no
    dcwi0=np.nan
    lencw=nfdx-nidx
    mxdT=np.nan
    try: 
        sst0=np.mean(dt[pn,nidx-1-4:nidx-1,0])
    except:
        sst0=np.mean(dt[pn,:nidx-1,0])

    
    # test if lenght of wind event is longer than 0... if not, no coldwake to calculate
    if nfdx-nidx>0:
        # find if there is really a cold wake. look for 5 days
        of=0
        ok=0
        while np.logical_and(ok==0, of<6):
            # is there a change in temp before begining of storm and previous day
            ct=dt[pn,nidx+of,0]-sst0 # nidx first day of "cooling" (the day the storm begins)
            # is there a cold wake?
            if ct > 0: # no decrease in temp, go for the next day, recalculate SST0
                of=of+1
                try: 
                    sst0=np.mean(dt[pn,nidx+of-1-4:nidx+of-1,0])
                except:
                    sst0=np.mean(dt[pn,:nidx+of-1,0])
            else:  # decrease, so start counting... but only if change is low
                #print('Cold wake') # cooling start the day after the storm begins (nidx+1)
                cow=1
                ok=1

        # if coldwake
        if cow==1:
            #print(of)

            # for the next days, calculate (#days), cumulative SSTa (from SST0), and max SSTa until the diff is < ?? 0.1
            dcwi0=nidx+of # first day of cold wake (for now same as arriving o f storm)
            dcwi=nidx+of # index to count days of cold wake
            lencw=1 # length of cold wake
            mxdT=dt[pn,dcwi,0]-sst0 # max change in temp during the wake

            # first day doesn't get evaluated (already a change) 
            dcwi=dcwi+1
            ok=0
            mxok=0
            while np.logical_and(ok==0,lencw<15):  
                # "anomaly stil larger than 0.1C, or the cw last less than 10days)
                if dt[pn,dcwi,0]<sst0-0.1:
                    # if temp keeps decreasing check for max change
                    if np.logical_and(dt[pn,dcwi,0]<dt[pn,dcwi-1,0],mxok==0):
                        if dt[pn,dcwi,0]-sst0<mxdT: # is today colder than the min SST from before?
                            mxdT=dt[pn,dcwi,0]-sst0 # min SST 
                        else:
                            mxok=1 # look for further for mx change
                    dcwi=dcwi+1
                    lencw=lencw+1
                else: # end of cold wake
                    ok=1
                    # length remain the same
        
    return cow,dcwi0,lencw,sst0, mxdT


In [ ]:
# calculate cold wake and create mask
plat=[]
plon=[]
pxdT=[] # max cooling
psst0=[] # initial (base) temperature

for pn in range(len(sif)):
    # calculate wind speed
    spd=np.sqrt(dt[pn,:,1]*dt[pn,:,1]+dt[pn,:,2]*dt[pn,:,2])
    # find actual arrival of storm date and wind at the time
    tw0,twf, clod=find_storm_beginend(pn)
    w0=spd[tw0]
   
    ## closest latitude in storm to point
    clolat=np.abs(storm['lat'][0].values - sif['lat'][pn]).argmin()
    # also get rid of points to close to the begining of the storm (8 points ~ 2 days)
    if np.logical_or(storm['lon'][0].values[clolat]-sif['lon'][pn]>1,clolat<9): ## !!! review this for southern hemisphere
        # no cold wake if more than 1 degree to the left of the storm 
        cow=0
    else:
        # is there a cold wake? and quantify
        cow, tcw0, lcw, sst0, xdT = cold_wake(pn,tw0,twf)    
    
    if cow==1:
        if np.logical_and(xdT<=-1,lcw>5): # threshold for cold wake
            pxdT.append(xdT) # max cooling
            psst0.append(sst0)
            
            # from storm info file
            plat.append(sif['lat'][pn])
            plon.append(sif['lon'][pn])


In [ ]:
# create a mask
coldwake=np.full((len(dtd.lat.values),len(dtd.lon.values)),np.nan)
coldwake_mask=np.full((len(dtd.lat.values),len(dtd.lon.values)),0.0)
for i in range(len(plat)):
    a=np.where(dtd.lat.values==plat[i])
    b=np.where(dtd.lon.values==plon[i])
    coldwake[a[0][0],b[0][0]]=pxdT[i]
    coldwake_mask[a[0][0],b[0][0]]=1


In [ ]:
cm=plt.cm.get_cmap('seismic')
plt.figure(figsize=(14,6))
x,y=np.meshgrid(dtd.lon,dtd.lat)
plt.subplot(121)
plt.scatter(x,y,c=coldwake,s=15, alpha=0.8, cmap=cm)
cb=plt.colorbar()
cb.set_label('Max Cooling')
plt.xlabel('Lon')
plt.ylabel('Lat')
plt.scatter(storm.lon.values,storm.lat.values,c='k',s=20, marker='*')

cm=plt.cm.get_cmap('seismic')
plt.subplot(122)
plt.scatter(x,y,c=coldwake_mask,s=15, alpha=0.8, cmap=cm)
cb=plt.colorbar()
cb.set_label('Mask')
plt.xlabel('Lon')
plt.ylabel('Lat')
plt.scatter(storm.lon.values,storm.lat.values,c='k',s=20, marker='*')

plt.show()